In [45]:
from itertools import product

import numpy as np
import pandas as pd
from scipy import stats as stats

import wandb

wandb.login()
api = wandb.Api()

In [4]:
tiny_lora_weights = pd.DataFrame(api.run("pico-lm/pico-relora/ixdq4sof").scan_history())
small_lora_weights = pd.DataFrame(api.run("pico-lm/pico-relora/a8nvp3mi").scan_history())
baseline_tiny_weights = pd.DataFrame(api.run("pico-lm/pico-relora/e8rwqdwk").scan_history())
baseline_small_weights = pd.DataFrame(api.run("pico-lm/pico-relora/evrk6fbj").scan_history())

In [35]:
def get_across_ranges(df: pd.DataFrame, metric_name: str, step_mod: int = 1, *ranges):
    if not ranges:
        ranges = (range(12),)
    combos = product(*ranges)
    mask = df["_step"] % step_mod == 0
    df = df[mask]
    x = [df[metric_name.format(*combo)].replace("NaN", np.nan).to_numpy() for combo in combos]
    return np.column_stack(x)

# Sparsity

In [ ]:
for met in ("hoyer", "gini"):
    for scale in ("tiny", "small"):
        metrstr = f"{met}_val/_forward_module.module.layers.{{}}.ov_circuit.base.weights"
        x = get_across_ranges(
            small_lora_weights if scale == "small" else tiny_lora_weights, metrstr, step_mod=20_000
        ).T

        r = stats.spearmanr(x[:, 1], np.arange(12)).statistic
        print(f"{met.title()} @ {scale.title()}: r = {r:.5f}")

Hoyer @ Tiny: r = 0.81818
Hoyer @ Small: r = 0.86014
Gini @ Tiny: r = 0.70878
Gini @ Small: r = 0.89123


# Step smoothness

In [ ]:
def evenness_metric(x: np.ndarray):
    diffs = np.diff(x, n=1, axis=-1)
    diffs_sigma = np.std(diffs, axis=-1)
    diffs_mu = np.mean(np.abs(diffs), axis=-1)

    variation_coeff = diffs_sigma / (diffs_mu + 1e-10)

    evenness = np.exp(-variation_coeff)

    return evenness

In [645]:
for metr_name in ("hoyer", "gini", "per", "norm", "condition_number"):
    for model_size in ("tiny", "small"):
        metrstr = f"{metr_name}_val/_forward_module.module.layers.{{}}.ov_circuit.base.weights"
        metr = get_across_ranges(
            small_lora_weights if model_size == "small" else tiny_lora_weights, metrstr, step_mod=2_000
        ).T

        fst = metr[:, 0]
        lst = metr[:, -1]

        evenness = evenness_metric(metr)

        # print(np.argmax(cv), np.argmin(cv))
        a = np.arange(12)
        even_r = stats.spearmanr(evenness, a).statistic

        inc_r = stats.spearmanr(lst - fst / fst, a).statistic
        fin_r = stats.spearmanr(lst, a).statistic

        print(f"{metr_name.title()} @ {model_size.title()}:")
        print(f"even r = {even_r:.5f}; inc r = {inc_r:.5f}, fin r = {fin_r:.5f}")

Hoyer @ Tiny:
even r = 0.88811; inc r = 0.81818, fin r = 0.81818
Hoyer @ Small:
even r = 0.78322; inc r = 0.86014, fin r = 0.86014
Gini @ Tiny:
even r = 0.52448; inc r = 0.70878, fin r = 0.70878
Gini @ Small:
even r = 0.86713; inc r = 0.89123, fin r = 0.89123
Per @ Tiny:
even r = -0.44056; inc r = -0.76923, fin r = -0.76923
Per @ Small:
even r = 0.01399; inc r = -0.96503, fin r = -0.96503
Norm @ Tiny:
even r = 0.46154; inc r = 0.97902, fin r = 0.97902
Norm @ Small:
even r = 0.53147; inc r = 0.97723, fin r = 0.97723
Condition_Number @ Tiny:
even r = -0.39161; inc r = 0.74825, fin r = 0.74825
Condition_Number @ Small:
even r = 0.74126; inc r = 0.96503, fin r = 0.96503
